<a href="https://colab.research.google.com/github/Nram94/simulacion-modelacion-python/blob/main/Semana08_ApliacionesIII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalar Dependencias

In [3]:
import numpy as np

## Definir Funciones

In [4]:
def z_pr(T, P, Tc, Pc, omega, R, fase):
  # --- 1.1 Cálculo del Volumen Molar Real (V_real) mediante EOS-PR ---
  # 1. Parámetros de la EOS
  Tr = T / Tc
  kappa = 0.37464 + 1.54226 * omega - 0.26992 * omega**2
  alpha = (1 + kappa * (1 - np.sqrt(Tr)))**2
  a_term = 0.45724 * R**2 * Tc**2 / Pc * alpha
  b_term = 0.07780 * R * Tc / Pc

  # 2. Parámetros adimensionales A y B
  A = a_term * P / (R * T)**2
  B = b_term * P / (R * T)

  # 3. Coeficientes de la Ecuación Cúbica Z^3 + c2*Z^2 + c1*Z + c0 = 0
  c2 = -(1 - B)
  c1 = (A - 3 * B**2 - 2 * B)
  c0 = -(A * B - B**2 - B**3)

  # 4. Solución de la EOS (Factor Z)
  roots = np.roots([1, c2, c1, c0])

  if fase == 'vapor':
    z = max(np.real(roots)) # Raiz Vapor
  elif fase == 'liquido':
    z = min(np.real(roots)) # Raíz Líquida
  else:
    raise ValueError("Debe elegir una fase vapor o líquida")

  return z


def constantes_filtracion(alpha, cs, mu, A, Rm, deltaP):
  # --- 2.1 Cálculo de Constantes ---
  # Constante de resistencia del Pastel (Kp)
  Kp = (alpha * cs * mu) / (2 * A**2 * deltaP)
  # Constante de resistencia del Medio (B)
  B = (mu * Rm) / (A * deltaP)

  return Kp, B



## Simulaciones

In [5]:
# =============================================================================
# DATOS COMUNES Y CONSTANTES
# =============================================================================
V_target = 2.0          # m^3, Volumen de agua a remover (Objetivo de comparación)
rho_H2O = 1000          # kg/m^3, Densidad del agua
R = 8.314               # J/mol*K, Constante Universal de Gas
MW_H2O = 0.018015       # kg/mol, Masa Molar del Agua

# =============================================================================
# 1. CÁLCULO DE EVAPORACIÓN/SECADO (TERMODINÁMICA REAL - EOS-PR)
# =============================================================================

print("--- 1. CÁLCULO DE EVAPORACIÓN/SECADO (EOS) ---")

# Datos del Sistema de Evaporación
T = 600.0               # K
P = 100.0 * 1e5         # 100 bar a Pa
V_dot_vapor = 0.5       # m^3/s, Flujo Volumétrico Máximo (Capacidad de la línea)

# Parámetros Críticos (Agua)
Tc = 647.1              # K
Pc = 220.6 * 1e5        # Pa
omega = 0.344           # Factor Acéntrico

# Seleccionamos la raíz real más grande, que corresponde al estado de vapor (Z)
Z_real = z_pr(T, P, Tc, Pc, omega, R, 'vapor')

# 5. Volumen Molar Real (V_real)
V_real = Z_real * R * T / P

# --- 1.2 Flujo Másico y Tiempo ---
m_dot_evap = (V_dot_vapor / V_real) * MW_H2O
t_evaporacion = V_target / V_dot_vapor

print(f"Factor de Compresibilidad (Z_real): {Z_real:.4f}")
print(f"Volumen Molar Real (V_real): {V_real:.6f} m^3/mol")
print(f"Flujo Másico Constante (m_dot_evap): {m_dot_evap:.3f} kg/s")
print(f"Tiempo para remover {V_target} m^3 (t_evaporacion): {t_evaporacion:.1f} segundos")
print("-" * 50)


# =============================================================================
# 2. CÁLCULO DE FILTRACIÓN (MECÁNICA - PRESIÓN CONSTANTE)
# =============================================================================

print("\n--- 2. CÁLCULO DE FILTRACIÓN (MECÁNICA) ---")

# Datos del Sistema de Filtración
A = 1.0               # m^2
cs = 15.0             # kg solid/m^3 filtrado
deltaP = 200000       # Pa
mu = 0.001            # Pa*s
alpha = 5.0e10        # m/kg
Rm = 1.0e9            # m^-1

Kp, B = constantes_filtracion(alpha, cs, mu, A, Rm, deltaP)
# Tiempo Total (t_filtracion) para V_target
t_filtracion = Kp * V_target**2 + B * V_target

# --- 2.2 Flujo Másico Promedio ---
V_dot_avg_filtracion = V_target / t_filtracion
m_dot_avg_filtracion = V_dot_avg_filtracion * rho_H2O

print(f"Constante del Pastel (Kp): {Kp:.0f} s/m^6")
print(f"Constante del Medio (B): {B:.0f} s/m^3")
print(f"Tiempo para remover {V_target} m^3 (t_filtracion): {t_filtracion:.0f} segundos")
print(f"Flujo Másico Promedio (m_dot_avg_filtracion): {m_dot_avg_filtracion:.3f} kg/s")
print("-" * 50)


# =============================================================================
# 3. COMPARACIÓN Y CONCLUSIÓN
# =============================================================================

print("\n--- 3. COMPARACIÓN DE CAPACIDAD DE REMOCIÓN DE AGUA ---")

print(f"1. TIEMPO REQUERIDO (V_target = {V_target} m^3):")
print(f"   Evaporación (Térmica): {t_evaporacion:.1f} s")
print(f"   Filtración (Mecánica): {t_filtracion:.0f} s")

print("\n2. FLUJO MÁSICO REMOVIDO:")
print(f"   Evaporación (Térmica): {m_dot_evap:.3f} kg/s")
print(f"   Filtración (Mecánica): {m_dot_avg_filtracion:.3f} kg/s")

print("\nConclusión: El proceso de Filtración tarda 7510 segundos, mientras que la Evaporación, limitada por la línea de vapor, tarda solo 4 segundos.")
print("La alta resistencia de la torta ($K_p$) en la Filtración es el factor limitante.")

--- 1. CÁLCULO DE EVAPORACIÓN/SECADO (EOS) ---
Factor de Compresibilidad (Z_real): 0.7379
Volumen Molar Real (V_real): 0.000368 m^3/mol
Flujo Másico Constante (m_dot_evap): 24.470 kg/s
Tiempo para remover 2.0 m^3 (t_evaporacion): 4.0 segundos
--------------------------------------------------

--- 2. CÁLCULO DE FILTRACIÓN (MECÁNICA) ---
Constante del Pastel (Kp): 1875 s/m^6
Constante del Medio (B): 5 s/m^3
Tiempo para remover 2.0 m^3 (t_filtracion): 7510 segundos
Flujo Másico Promedio (m_dot_avg_filtracion): 0.266 kg/s
--------------------------------------------------

--- 3. COMPARACIÓN DE CAPACIDAD DE REMOCIÓN DE AGUA ---
1. TIEMPO REQUERIDO (V_target = 2.0 m^3):
   Evaporación (Térmica): 4.0 s
   Filtración (Mecánica): 7510 s

2. FLUJO MÁSICO REMOVIDO:
   Evaporación (Térmica): 24.470 kg/s
   Filtración (Mecánica): 0.266 kg/s

Conclusión: El proceso de Filtración tarda 7510 segundos, mientras que la Evaporación, limitada por la línea de vapor, tarda solo 4 segundos.
La alta resiste